### Imports et création BDD

In [1]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

In [2]:
# Connexion moteur SQLite et création BDD
engine = create_engine('sqlite:///colleges.db',echo=True)

### Création tables

In [3]:
# Objet Metadata
meta = MetaData()

In [4]:
# Création table parcours
parcours = Table("parcours",meta,
                 Column("id",Integer,primary_key=True),
                 Column("name",String))

meta.create_all(engine)

2025-06-29 23:32:12,674 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:32:12,675 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parcours")
2025-06-29 23:32:12,675 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:32:12,677 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("parcours")
2025-06-29 23:32:12,678 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:32:12,680 INFO sqlalchemy.engine.Engine 
CREATE TABLE parcours (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2025-06-29 23:32:12,681 INFO sqlalchemy.engine.Engine [no key 0.00095s] ()
2025-06-29 23:32:12,778 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
# Création table students avec clé étrangère sur parcours
students = Table("students",meta,
                 Column("id",Integer,primary_key=True),
                 Column("firstname",String),
                 Column("lastname",String),
                 Column("parcours_id",Integer,ForeignKey("parcours.id"))
                )

meta.create_all(engine)

2025-06-29 23:32:14,658 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:32:14,659 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parcours")
2025-06-29 23:32:14,660 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:32:14,661 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2025-06-29 23:32:14,662 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:32:14,664 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2025-06-29 23:32:14,665 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:32:14,667 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	firstname VARCHAR, 
	lastname VARCHAR, 
	parcours_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(parcours_id) REFERENCES parcours (id)
)


2025-06-29 23:32:14,669 INFO sqlalchemy.engine.Engine [no key 0.00162s] ()
2025-06-29 23:32:14,754 INFO sqlalchemy.engine.Engine COMMIT


### Insertion de lignes

In [6]:
values1 = [{"id":1, "name":"Data Engineering"},{"id":2, "name":"Data Science"}]

values2 = [{"id":1, "firstname":"Jean","lastname":"Bart", "parcours_id":2},{"id":2, "firstname":"Edmond","lastname":"Dantes", "parcours_id":1}]

In [7]:
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'exécuter une transaction
        try:
            
            # On utilise le langage SQL en format texte
            ins = 'INSERT INTO {tablename} VALUES (:id, :name)'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=parcours.name)

            # transformer la chaîne en instruction SQLAlchemy
            stmt = text(ins)
           
            # Enfin on peut utiliser les dicos créés en exécutant la commande SQL
            connection.execute(stmt, values1)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

2025-06-29 23:32:19,305 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:32:19,306 INFO sqlalchemy.engine.Engine INSERT INTO parcours VALUES (?, ?)
2025-06-29 23:32:19,308 INFO sqlalchemy.engine.Engine [generated in 0.00151s] [(1, 'Data Engineering'), (2, 'Data Science')]
2025-06-29 23:32:19,310 INFO sqlalchemy.engine.Engine COMMIT


### Méthode plus directe

In [ ]:
# On utilise le langage SQL en format texte
ins = 'INSERT INTO {tablename} VALUES (:id, :firstname, :lastname, :parcours_id)'
            
# On précise ce format particulier grâce à la fonction membre format
ins = ins.format(tablename=students.name)

# transformer la chaîne en instruction SQLAlchemy
stmt = text(ins)
           
# Fera automatiquement le COMMIT si ok, le ROLLBACK sinon
with engine.begin() as connection:
    connection.execute(stmt, values2)

2025-06-29 23:38:43,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:38:43,153 INFO sqlalchemy.engine.Engine INSERT INTO students VALUES (?, ?, ?, ?)
2025-06-29 23:38:43,154 INFO sqlalchemy.engine.Engine [cached since 329.1s ago] [(1, 'Jean', 'Bart', 2), (2, 'Edmond', 'Dantes', 1)]
2025-06-29 23:38:43,156 INFO sqlalchemy.engine.Engine COMMIT


### Inspector

In [10]:
inspector = inspect(engine)

In [11]:
# Liste des tables
inspector.get_table_names()

2025-06-29 23:39:01,114 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:39:01,115 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-06-29 23:39:01,115 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:39:01,117 INFO sqlalchemy.engine.Engine ROLLBACK


['parcours', 'students']

In [12]:
# Liste des colonnes d'une table
inspector.get_columns(table_name='students')

2025-06-29 23:39:02,729 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:39:02,730 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("students")
2025-06-29 23:39:02,730 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:39:02,732 INFO sqlalchemy.engine.Engine ROLLBACK


[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'primary_key': 1},
 {'name': 'firstname',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'lastname',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'parcours_id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

In [13]:
# Liste des clés étrangères d'une table
inspector.get_foreign_keys(table_name='students')

2025-06-29 23:39:04,224 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:39:04,225 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("students")
2025-06-29 23:39:04,226 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:39:04,228 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2025-06-29 23:39:04,228 INFO sqlalchemy.engine.Engine [raw sql] ('students',)
2025-06-29 23:39:04,232 INFO sqlalchemy.engine.Engine ROLLBACK


[{'name': None,
  'constrained_columns': ['parcours_id'],
  'referred_schema': None,
  'referred_table': 'parcours',
  'referred_columns': ['id'],
  'options': {}}]

### Lecture de lignes

In [14]:
with engine.begin() as connection:
    results = connection.execute(text("SELECT * FROM students;"))
    print(results.fetchall())

with engine.begin() as connection:
    results = connection.execute(text("SELECT * FROM parcours;"))
    print(results.fetchall())


2025-06-29 23:39:24,323 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:39:24,324 INFO sqlalchemy.engine.Engine SELECT * FROM students;
2025-06-29 23:39:24,326 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ()
[(1, 'Jean', 'Bart', 2), (2, 'Edmond', 'Dantes', 1)]
2025-06-29 23:39:24,327 INFO sqlalchemy.engine.Engine COMMIT
2025-06-29 23:39:24,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:39:24,329 INFO sqlalchemy.engine.Engine SELECT * FROM parcours;
2025-06-29 23:39:24,329 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ()
[(1, 'Data Engineering'), (2, 'Data Science')]
2025-06-29 23:39:24,331 INFO sqlalchemy.engine.Engine COMMIT


### Suppression de tables

In [15]:
with engine.begin() as connection:
    results = connection.execute(text("DROP TABLE IF EXISTS students;"))

2025-06-29 23:39:52,836 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:39:52,837 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS students;
2025-06-29 23:39:52,838 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ()
2025-06-29 23:39:52,955 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
inspector = inspect(engine)
inspector.get_table_names()

2025-06-29 23:40:30,400 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-29 23:40:30,401 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-06-29 23:40:30,402 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-29 23:40:30,404 INFO sqlalchemy.engine.Engine ROLLBACK


['parcours']